In [12]:
import warnings
warnings.filterwarnings("ignore")
import torch
import sys

import os
import numpy as np
from tqdm import tqdm
import torchaudio
import librosa
import glob

sys.path.append("../../cuhksz-phd/sho_util/pyfiles/")
from pytorch import cuda2numpy, cuda2cpu
from basic import plot_spectrogram
from sound import play_audio

sys.path.append("../")
from pyfiles.processsound import trim_audio_and_save

import json
sys.path.append("./../../hifi-gan/")
from meldataset import mel_spectrogram, load_wav
from env import AttrDict, build_env
config = f'../../hifi-gan/config_16kHz.json'
with open(config) as f:
    data = f.read()
json_config = json.loads(data)
h = AttrDict(json_config)

In [70]:
def get_mel_spectrogram(path, MAX_WAV_VALUE=32768.0):
    audio, _ = load_wav(path, h.sampling_rate)
    if audio.max()>1.0:
        audio = audio / MAX_WAV_VALUE
    audio = torch.FloatTensor(audio)
    audio = audio.unsqueeze(0)
    mel = mel_spectrogram(audio, h.n_fft, h.num_mels,
                          h.sampling_rate, h.hop_size, h.win_size, h.fmin, h.fmax,
                          center=False)
    mel = np.array(mel[0])
    return mel

In [71]:
dataset_dir = "/mntcephfs/lee_dataset/tts/LibriTTS_R/"
# feat_base_dir = "/mntcephfs/lab_data/shoinoue/Dataset/LibriTTS_R/features/"
feat_base_dir = "/mntcephfs/data/audiow/shoinoue/Dataset/LibriTTS_R/features/"
fs = 16000

speakers = [os.path.basename(a) for a in glob.glob(dataset_dir + "*/*")]
speakers.sort()

In [77]:
save = True
tempfile = "temp3.wav"
trim_threshold_in_db = 30

files = []
for spk in tqdm(speakers):
    files += glob.glob(dataset_dir + f"*/{spk}/*/*.wav")
files.sort()
for path in tqdm(files):
    trim_audio_and_save(path, savepath=tempfile, trim_threshold_in_db=trim_threshold_in_db)
    mel = get_mel_spectrogram(tempfile)
    savepath = feat_base_dir + "/".join(path.split("/")[-4:])[:-4] + "_hifiganmel.npy"
    if save:
        np.save(savepath, mel)

  0%|          | 46/160267 [00:03<3:19:10, 13.41it/s]

KeyboardInterrupt



 Normalize

In [81]:
from sklearn.preprocessing import StandardScaler
import numpy as np
import joblib

In [82]:
scaler_name = "LibriTTS-R_hifiganmel"

In [86]:
scaler = StandardScaler()
files = []
for spk in tqdm(speakers):
    files += glob.glob(feat_base_dir + f"train*/{spk}/*/*_hifiganmel.npy")
files.sort()
for path in tqdm(files):
    if "km" in path:
        continue
    mel = np.load(path)
    scaler.partial_fit(mel.T)
            
scaler_filename = f"ckpts/scalers/{scaler_name}.save"
joblib.dump(scaler, scaler_filename) 
# a = joblib.load(scaler_filename)

100%|██████████| 46/46 [00:00<00:00, 496.02it/s]


['ckpts/scalers/LibriTTS-R_hifiganmel.save']